In [2]:
import sys
sys.path.append(r'..')
from data import read_svm_data
from cvxopt import matrix, solvers
import numpy as np

In [3]:
training_labels, training_images = read_svm_data("training", r"../../MNIST_ORG", [2, 3, 8, 9])
testing_labels, testing_images = read_svm_data("testing", r"../../MNIST_ORG", [2, 3, 8, 9])

training_images.shape, training_labels.shape, testing_images.shape, testing_labels.shape

((20000, 784), (20000,), (3974, 784), (3974,))

In [4]:
training_images = training_images.astype(np.float64) / 255.0
testing_images = testing_images.astype(np.float64) / 255.0

In [5]:
C = 100
N = training_labels.shape[0]  # number of training samples
d = training_images.shape[1]  # dimension of each sample
labels_to_classify = [2, 3, 8, 9]
classifiers = {}

In [6]:
def polynomial_kernel(x, y, degree=2, alpha=1, beta=1):
    return (alpha * np.dot(x, y) + beta) ** degree

In [ ]:
K = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        K[i, j] = polynomial_kernel(training_images[i], training_images[j])

In [ ]:
# Set up the matrices for the QP problem
P = matrix(np.outer(training_labels, training_labels) * K)
q = matrix(-np.ones(N))
G = matrix(np.vstack((-np.eye(N), np.eye(N))))
h = matrix(np.hstack((np.zeros(N), np.ones(N) * C)))
A = matrix(training_labels, (1, N), 'd')
b = matrix(0.0)

# Solve QP problem
solution = solvers.qp(P, q, G, h, A, b)
alphas = np.ravel(solution['x'])

# Extract support vectors
support_vector_indices = alphas > 1e-5
alphas = alphas[support_vector_indices]
support_vectors = training_images[support_vector_indices]
support_vector_labels = training_labels[support_vector_indices]

# Calculate b
b = np.mean([y_k - np.sum(alphas * support_vector_labels * K[ind, support_vector_indices])
             for ind, y_k in enumerate(support_vector_labels)])

# Prediction using the kernel function
def predict(X):
    return np.sign(np.sum(alphas * support_vector_labels * polynomial_kernel(support_vectors, X, degree=2, coef0=1), axis=1) + b)

# Evaluate model
training_predictions = predict(training_images)
testing_predictions = predict(testing_images)

# Calculate accuracies
training_accuracy = np.mean(training_predictions == training_labels)
testing_accuracy = np.mean(testing_predictions == testing_labels)

print(f"Training Accuracy: {training_accuracy * 100:.2f}%")
print(f"Testing Accuracy: {testing_accuracy * 100:.2f}%")